<a href="https://colab.research.google.com/github/Ludvins/Practicas_PDGE/blob/master/CUDA/Suma_y_Stencil1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Información del sistema
Comprobamos las características del sistema que nos ha proporcionado Google Colab. Utilizando `lscpu` podemos obtener diversas características dela CPU que utiliza el sistema.

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.000
BogoMIPS:            4400.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

El comando `free` nos permite conocer información sobre la memoria física y swap del sistema. Utilizamos el flag `-h` para indicar que buscamos tener la salida en un formato más leible (Megabytes, Gygabytes...)

In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        538M         10G        972K        2.0G         11G
Swap:            0B          0B          0B


Verificamos la versión de Cuda instalada:

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


Utilizamos la interzaz de configuración de sistemas de NVIDIA (NVIDIA System Management Interface) para conocer el estado de la tarjeta gráfica que vamos a utilizar.

In [ ]:
!nvidia-smi

Fri Oct 16 15:19:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Podemos ejecutar el codigo de ejemplo presente en la libreria de Cuda que nos enumera las propeidades del dispositivo Cuda que existe en el sistema. Para ello, cambiamos de directorio a aquel donde se encuentra el codigo fuente:


In [ ]:
%cd /usr/local/cuda/samples/1_Utilities/deviceQuery/

/usr/local/cuda-10.1/samples/1_Utilities/deviceQuery


El programa no se encuentra compilado, pero provee de un makefile para su facil compilación:

In [ ]:
!make

/usr/local/cuda-10.1/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda-10.1/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o deviceQuery deviceQuery.o 
mkdir -p ../../bin/x86_64/linux/release
cp deviceQuery ../../bin/x86_64/linux/rele

Si ejecutamos el programa, obtenemos información interesante, como que el número máximo de threads por bloque que podemos utilizar es 1024:
```
Maximum number of threads per block: 1024
```
También vemos la dimención máxima que podemos dar a cada dimensión de bloque y grid:
```
Max dimension size of a thread block (x,y,z): (1024, 1024, 64),
Max dimension size of a grid size    (x,y,z): (2147483647, 65535, 65535)
```

In [ ]:
!!./deviceQuery

['./deviceQuery Starting...',
 '',
 ' CUDA Device Query (Runtime API) version (CUDART static linking)',
 '',
 'Detected 1 CUDA Capable device(s)',
 '',
 'Device 0: "Tesla P100-PCIE-16GB"',
 '  CUDA Driver Version / Runtime Version          10.1 / 10.1',
 '  CUDA Capability Major/Minor version number:    6.0',
 '  Total amount of global memory:                 16281 MBytes (17071734784 bytes)',
 '  (56) Multiprocessors, ( 64) CUDA Cores/MP:     3584 CUDA Cores',
 '  GPU Max Clock rate:                            1329 MHz (1.33 GHz)',
 '  Memory Clock rate:                             715 Mhz',
 '  Memory Bus Width:                              4096-bit',
 '  L2 Cache Size:                                 4194304 bytes',
 '  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)',
 '  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers',
 '  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers',

# Ejercicio 1

El código de ejemplo en suma de los elementos de un vector realiza la
suma de dos vectores en la GPU.
1. Comente los diferentes casos propuestos en el ejemplo y
conteste a las preguntas.
2. Se propone extender el código de este ejemplo para que realice
la resta (o suma) de matrices cuadradas de dimensión N. 
  - Configure adecuadamente el Grid de threads para aceptar
matrices de cualquier tamaño.
  - En el kernel, utilice las variables blockIdx y threadIdx
adecuadamente para acceder a una estructura bidimensional.

### Suma de Vectores completa

Vemos a continuación el código concreto que permite sumar dos vectores guardando el resultado en el segundo de estos.

En él, utilizamos un total de 512 threads por bloque y (N + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK bloques distintos. De esta forma, si N es multiplo de THREADS_PER_BLOCK
$$
\frac{N + TPB - 1}{TPB} = \frac{N}{TPB} + 1 - \frac{1}{TPB} = K + 1 -\frac{1}{TPB}
$$
Lo cual tras transformarlo a entero resultaria en $K = N/TPB$. En caso de no ser multiplos, $N$ se descompone como $N_1 + N_2$ donde $N_1$ si es multiplo de $TPB$ y $0 < N_2 < TPB$:
$$
\frac{N + TPB - 1}{TPB} = \frac{N_1}{TPB} + \frac{N_2}{TPB} + 1 - \frac{1}{TPB} = K + 1 + \frac{N_2}{TPB} -\frac{1}{TPB}
$$
Por ser $0 < N_2 < TPB$, luego 
$$
\frac{N_2}{TPB} -\frac{1}{TPB} \geq 0
$$
Resultando en $K + 1$ iteraciones.

In [ ]:
%%writefile suma_vec.cu
#include <iostream>
#include <math.h>

// Define array size
#define N (1024*1024)
// Define number of threads per block
#define THREADS_PER_BLOCK 512

// Función de suma de vectores en GPU.
__global__ void add(float *x, float *y) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    // Comprobamos que el índice es correcto
    if (i < N )
        y[i]=x[i]+y[i];
}


// Función principal (CPU)
int main(void) {
    // Arrays en CPU
    float *x, *y;                 // = new float[N];
    // Arrays en GPU
    float *d_x, *d_y;             // = new float[N]; 
    // Tamaño de los arrays
    int size = N*sizeof(float);

    // Reservamos memoria en el dispositivo.
    cudaMalloc((void **)&d_x, size);
    cudaMalloc((void **)&d_y, size);

    // Reservamos memoria en el host
    x = (float *)malloc(size);
    y = (float *)malloc(size);

    // Inicializamos los vectores
    for (int i =0; i < N; i++ ){
        x[i]= 1.0f;
        y[i]= 2.0f;
    }

    // Copiamos los valores al dispositivo
    cudaMemcpy(d_x, x, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_y, y, size, cudaMemcpyHostToDevice);

    // Lanzamos add
    add<<<(N + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK, THREADS_PER_BLOCK>>>(d_x, d_y);

    // Esperamos que se realicen todas las iteraciones.
    cudaDeviceSynchronize();

    // Copiamos el resultado al host
    cudaMemcpy(y, d_y, size, cudaMemcpyDeviceToHost);
    
    // Comprobamos los resultados
    float maxError = 0.0f;
    int contError = 0;
   
    for (int i=0; i <N; i++){
        maxError=fmax(maxError,fabs(y[i]-3.0f));
        if (y[i] != 3.0) contError++; 
    }
    std::cout << "suma de " << N << " Elementos" << std::endl;
    std::cout << "Número de Errores: " <<contError << std::endl;
    std::cout << "Max error: " <<maxError << std::endl;

    free(x); free(y);
    cudaFree(d_x); cudaFree(d_y);
   return 0;
}

Overwriting suma_vec.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma_vec.cu -o suma_vec -lcudadevrt

In [ ]:
!./suma_vec

suma de 1048576 Elementos
Número de Errores: 0
Max error: 0


In [ ]:
!nvprof ./suma_vec

==1124== NVPROF is profiling process 1124, command: ./suma_vec
suma de 4194304 Elementos
Número de Errores: 0
Max error: 0
==1124== Profiling application: ./suma_vec
==1124== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   58.64%  9.3680ms         1  9.3680ms  9.3680ms  9.3680ms  [CUDA memcpy DtoH]
                   40.77%  6.5130ms         2  3.2565ms  3.2527ms  3.2603ms  [CUDA memcpy HtoD]
                    0.59%  94.558us         1  94.558us  94.558us  94.558us  add(float*, float*, float*)
      API calls:   88.40%  154.30ms         3  51.433ms  88.906us  154.11ms  cudaMalloc
                   10.01%  17.474ms         3  5.8247ms  3.4203ms  10.582ms  cudaMemcpy
                    1.16%  2.0296ms         3  676.54us  167.66us  1.1382ms  cudaFree
                    0.21%  362.95us         1  362.95us  362.95us  362.95us  cuDeviceTotalMem
                    0.09%  165.22us        97  1.7030us     133ns  54.38

Si ejecutamos el programa utilizando un total de 256 threads por bloque y un solo bloque, es decir:
```
add<<<1, 256>>>(d_x, d_y);
```
Obtendríamos que solo se ha realizado la operación sobre 256 valores de los N posibles, esto mismo pasa utilizando 256 bloques y un thread en cada uno. 

Por otro lado, si ejecutamos dicha función sin quitar el bucle for que teníamos inicialmente en la función add:

```
__global__ void add(int n, float *x, float *y) {
    for (int i =0; i < n; i++ ){
        y[i]=x[i]+y[i];
    }
}
```
obtendríamos que cada thread lanzado realizaría todas las sumas, pudiendo ocurrir que se pisaran unas a otras

```
Suma de 1048576 elementos
Número de errores: 3473
Max error: 2
```

**Pendiente: Comentar <1,256> sin datarace y suma en la GPU con paralelismo de bloques.Prueba con mas Thread por bloque. Pruebas variando N y la complejidad de la
operación matemática.**



## Suma de matrices

In [6]:
%%writefile suma_mat.cu
#include <iostream>
#include <math.h>
// Definimos las dimensiones de las matrices
#define R 512
#define C 512

// __global__ indica que se ejecuta en el dispositivo, por lo tanto
// x, y,deben apuntar a memoria en el dispositivo.

__global__ void add(float *x, float *y) {
    
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    int index = i*R + j;
    if (index < R*C)
      y[index] = x[index] + y[index];
}


// Define number of threads per block
#define THREADS_PER_BLOCK 32

int main(void) {
    // Inicializamos las variables
    float *x, *y;                 // = new float[N];
    float *d_x, *d_y;             // = new float[N]; 
    // Calculamos el tamaño de las matrices
    int size = C*R*sizeof(float);

    // Reservamos memoria en el dispositivo.
    cudaMalloc((void **)&d_x, size);
    cudaMalloc((void **)&d_y, size);

    // Reservamos memoria en el host, trataremos las matrices como vectores
    // con la dimension correspondiente y accederemos a ellos usando 2 indices.
    x = (float *)malloc(R*C*sizeof(float*));
    y = (float *)malloc(R*C*sizeof(float*));

    // Inicializamos los datos
    for (int i =0; i < R; i++ ){
        for (int j = 0; j < C; j++){
            x[i*R + j] = 1.0f;
            y[i*R + j] = 2.0f;
        }
    }

    // Copiamos los valores al dispositivo
    cudaMemcpy(d_x, x, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_y, y, size, cudaMemcpyHostToDevice);

    // Creamos los objetos dim3 tanto para el Grid como para el bloque.
    dim3 dimGrid ( (R + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK , (C + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK ,1 ) ;
    // Definimos la dimension del bloque.
    dim3 dimBlock( THREADS_PER_BLOCK, THREADS_PER_BLOCK, 1 ) ;

    // Lamzamos la función en el dispositivo.
    add<<<dimGrid, dimBlock>>>(d_x, d_y);

    // Esperamos que se realicen todas las operaciones.
    cudaDeviceSynchronize();

    // Copiamos el resultado al host.
    cudaMemcpy(y, d_y, size, cudaMemcpyDeviceToHost);
    
    // Comprobamos los resultados
    float maxError = 0.0f;
    int contError = 0;
   
    for (int i=0; i <R; i++){
        for (int j = 0; j < C; j++){
          maxError=fmax(maxError,fabs(y[i*R + j]-3.0f));
          if (y[i*R + j] != 3.0) contError++; 
        }
    }
    std::cout << "Suma de " << R*C << " Elementos" << std::endl;
    std::cout << "Número de Errores: " << contError << std::endl;
    std::cout << "Max error: " << maxError << std::endl;

    // Liberamos la memoria tanto en el host como en el dispositivo.
    free(x); free(y);
    cudaFree(d_x); cudaFree(d_y);
    return 0;
}

Overwriting suma_mat.cu


In [7]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma_mat.cu -o suma_mat -lcudadevrt

In [8]:
!./suma_mat

suma de 262144 Elementos
Número de Errores: 0
Max error: 0


## Spencil

Comenzamos con la versión del spencil sin memoria compartida.

In [ ]:
%%writefile 1destencilexercise_v1.cu
#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) {
    // Indice global de la posicion central de los datos del spencil que va a usar el thread.
    int index = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;

    // Realizamos la operación del spencil
    int result = 0;
    for (int offset = -RADIUS; offset <= RADIUS; offset++)
        result += in[index + offset];

    // Guardamos el resultado
    out[index-RADIUS] = result;
}

int main() {
  unsigned int i;
  int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
  int *d_in, *d_out;

  // Incializamos los datos en el host
  for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
    h_in[i] = 1; // Con un valor de 1 y un radio de 3, el array de resultados deberia ser de 7's.

  // Reservamos memoria en el dispositivo
  cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
  cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

  // Copiamos los datos en la GPU
  cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

  // Aplicamos el spencil
  stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

  // Copiamos los resultados
  cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

  // Verificamos que se ha hecho correctamente
  for( i = 0; i < NUM_ELEMENTS; ++i )
    if (h_out[i] != 7)
    {
      printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
      break;
    }

  if (i == NUM_ELEMENTS)
    printf("SUCCESS!\n");

  // Liberamos memoria
  cudaFree(d_in);
  cudaFree(d_out);

  return 0;
}

Overwriting 1destencilexercise_v1.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 1destencilexercise_v1.cu -o 1destencilexercise -lcudadevrt
!./1destencilexercise

SUCCESS!


Comprobamos el comportamiento temporal

In [ ]:
!nvprof ./1destencilexercise

==575== NVPROF is profiling process 575, command: ./1destencilexercise
SUCCESS!
==575== Profiling application: ./1destencilexercise
==575== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   37.81%  4.4160us         1  4.4160us  4.4160us  4.4160us  [CUDA memcpy DtoH]
                   36.16%  4.2240us         1  4.2240us  4.2240us  4.2240us  [CUDA memcpy HtoD]
                   26.03%  3.0400us         1  3.0400us  3.0400us  3.0400us  stencil_1d(int*, int*)
      API calls:   99.61%  205.42ms         2  102.71ms  7.3560us  205.41ms  cudaMalloc
                    0.18%  367.52us         1  367.52us  367.52us  367.52us  cuDeviceTotalMem
                    0.07%  152.23us        97  1.5690us     145ns  64.354us  cuDeviceGetAttribute
                    0.06%  127.92us         2  63.961us  14.654us  113.27us  cudaFree
                    0.04%  84.581us         2  42.290us  33.218us  51.363us  cudaMemcpy
              

Al no utilizar memoria compartida, la mayoria del tiempo se dedica a copia de elementos.

**TODO**: 2. Use el generador de perfiles para determinar cuál es el problema.


## Versión con memoria compartida sin sincronización de threads

In [12]:
%%writefile 1destencilexercise_v2.cu
#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    // Reservamos la memoria compartida
    __shared__ int temp[BLOCK_SIZE + 2*RADIUS];
    // Indice central en el array global
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    // Indice centran en el array local
    int lindex = threadIdx.x + RADIUS;

    // Leemos los puntos a la memoria compartida
    temp[lindex] = in[gindex];
    if (threadIdx.x < RADIUS) 
    {
        temp[lindex - RADIUS] = in[gindex - RADIUS];
        temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }

    // Hacemos la operación
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += temp[lindex + offset];

    // Guardamos el resultado
    out[gindex-RADIUS] = result;
}

int main()
{

  // Ejecutamos el código 100 veces independientes
  for (int j = 0; j < 100; j++){
      unsigned int i;
      int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
      int *d_in, *d_out;

      // Inicializamos los datos
      for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
          h_in[i] = 1; 

      // Reserva memoria en la GPU
      cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
      cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

      // Copiar los datos al dispositivo
      cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

      stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

      cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

      for( i = 0; i < NUM_ELEMENTS; ++i )
        if (h_out[i] != 7)
        {
          printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
          break;
        }

      // Free out memory
      cudaFree(d_in);
      cudaFree(d_out);
      cudaDeviceReset();
  }

  return 0;
}



Overwriting 1destencilexercise_v2.cu


In [13]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 1destencilexercise_v2.cu -o 1destencilexercise -lcudadevrt

In [14]:
!./1destencilexercise

Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[63] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[95] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[95] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] ==

## Versión con memoria compartida y sincronización de threads

In [15]:
%%writefile 1destencilexercise_v3.cu
#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    __shared__ int temp[BLOCK_SIZE + 2*RADIUS];
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    int lindex = threadIdx.x + RADIUS;

    // Read input elements into shared memory
    temp[lindex] = in[gindex];
    if (threadIdx.x < RADIUS) 
    {
        temp[lindex - RADIUS] = in[gindex - RADIUS];
        temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }
    __syncthreads ();
    // Apply the stencil
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += temp[lindex + offset];

    // Store the result
    out[gindex-RADIUS] = result;
}

int main()
{

      unsigned int i;
      int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
      int *d_in, *d_out;

    // Initialize host data
    for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
      h_in[i] = 1; // With a value of 1 and RADIUS of 3, all output values should be 7

    // Allocate space on the device
    cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
    cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

    // Copy input data to device
    cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

    stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

    cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

    // Verify every out value is 7
    for( i = 0; i < NUM_ELEMENTS; ++i )
      if (h_out[i] != 7)
      {
        printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
        break;
      }
 
  if (i == NUM_ELEMENTS)
    printf("SUCCESS!\n");


    // Free out memory
    cudaFree(d_in);
    cudaFree(d_out);

  return 0;
}


Writing 1destencilexercise_v3.cu


In [16]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 1destencilexercise_v3.cu -o 1destencilexercise -lcudadevrt
!./1destencilexercise

SUCCESS!


In [17]:
!nvprof ./1destencilexercise

==881== NVPROF is profiling process 881, command: ./1destencilexercise
SUCCESS!
==881== Profiling application: ./1destencilexercise
==881== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   37.06%  4.3520us         1  4.3520us  4.3520us  4.3520us  [CUDA memcpy DtoH]
                   35.97%  4.2240us         1  4.2240us  4.2240us  4.2240us  [CUDA memcpy HtoD]
                   26.98%  3.1680us         1  3.1680us  3.1680us  3.1680us  stencil_1d(int*, int*)
      API calls:   99.61%  192.38ms         2  96.188ms  6.8640us  192.37ms  cudaMalloc
                    0.20%  382.92us         1  382.92us  382.92us  382.92us  cuDeviceTotalMem
                    0.07%  144.48us        97  1.4890us     164ns  59.230us  cuDeviceGetAttribute
                    0.05%  100.27us         2  50.132us  12.964us  87.301us  cudaFree
                    0.04%  71.811us         2  35.905us  30.655us  41.156us  cudaMemcpy
              